In [ ]:
import re
import json
import requests
import asyncio
from typing import Any, List, Mapping, Optional

import discord
from discord import app_commands
from discord.ext import commands
from discord.ext.commands import Bot

import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings
import uuid
import langchain
from langchain.chains import (
    ConversationChain,
    LLMChain,
    LLMMathChain,
    TransformChain,
    SequentialChain,
)
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.llms.base import LLM, Optional, List, Mapping, Any
from langchain.embeddings.openai import OpenAIEmbeddings
from textwrap import dedent
from langchain.memory import (
    ChatMessageHistory,
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
)
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.vectorstores import Chroma
from langchain.agents import load_tools
from langchain.agents import initialize_agent
import os
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv
from helpers.constants import MAINTEMPLATE, BOTNAME
from helpers.custom_memory import *
from pydantic import Field
from koboldllm import KoboldApiLLM
from ooballm import OobaApiLLM
from langchain.llms import TextGen




class Chatbot:

    def __init__(self, name):
        self.histories = {}  # Initialize the history dictionary
        self.stop_sequences = {}  # Initialize the stop sequences dictionary
        self.char_name = name
        self.memory = CustomBufferWindowMemory(k=10, ai_prefix=self.char_name)
        self.history = "[Beginning of Conversation]"

        self.template = MAINTEMPLATE

        self.PROMPT = PromptTemplate(
            input_variables=["history", "input"], template=self.template
        )
        self.conversation = ConversationChain(
            prompt=self.PROMPT,
            llm=self.llm,
            verbose=True,
            memory=self.memory,
        )

    # create doc string

    def get_memory_for_channel(self, channel_id):
        """Get the memory for the channel with the given ID. If no memory exists yet, create one."""
        if channel_id not in self.histories:
            self.histories[channel_id] = CustomBufferWindowMemory(
                k=20, ai_prefix=self.char_name
            )
            self.memory = self.histories[channel_id]
        return self.histories[channel_id]

    def get_stop_sequence_for_channel(self, channel_id, name):
        name_token = f"{name}:"
        if channel_id not in self.stop_sequences:
            self.stop_sequences[channel_id] = [
                "\n### Instruction:",
                "\n### Response:",
            ]  # EXPERIMENT: Testing adding the triple line break to see if that helps with stopping
        if name_token not in self.stop_sequences[channel_id]:
            self.stop_sequences[channel_id].append(name_token)
        return self.stop_sequences[channel_id]

    # this command will detect if the bot is trying to send  \nself.char_name: in its message and replace it with an empty string
    def detect_and_replace(self, message_content):
        if f"\n{self.char_name}:" in message_content:
            message_content = message_content.replace(f"\n{self.char_name}:", "")
        return message_content

    def generate_response(self, message, message_content) -> None:
        channel_id = str(message.channel.id)
        name = message.author.display_name
        memory = self.get_memory_for_channel(channel_id)
        stop_sequence = self.get_stop_sequence_for_channel(channel_id, name)
        print(f"stop sequences: {stop_sequence}")
        formatted_message = f"{name}: {message_content}"

        # Create a conversation chain using the channel-specific memory
        conversation = ConversationChain(
            prompt=self.PROMPT,
            llm=self.llm,
            verbose=True,
            memory=memory,
        )

        input_dict = {"input": formatted_message, "stop": stop_sequence}

        response_text = conversation(input_dict)

        response = self.detect_and_replace(response_text["response"])

        return response

    # this command receives a name, channel_id, and message_content then adds it to history
    def add_history(self, name, channel_id, message_content) -> None:
        # get the memory for the channel
        memory = self.get_memory_for_channel(str(channel_id))

        formatted_message = f"{name}: {message_content}"

        # add the message to the memory
        print(f"adding message to memory: {formatted_message}")
        memory.add_input_only(formatted_message)
        return None

    # receives a prompt from the user and an observation from the agent then sends to the LLM for a reply
    def agent_command(self, name, channel_id, prompt, observation) -> None:
        memory = self.get_memory_for_channel(channel_id)
        self.get_stop_sequence_for_channel(channel_id, name)
        stop_sequence = self.get_stop_sequence_for_channel(channel_id, name)
        formatted_user_message = f"{name}: {prompt}"
        formatted_bot_message = f"### Input: {observation}"
        AGENTTEMPLATE = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are Tensor, a lively and playful AI chatbot. You communicate in a modern, casual manner using contemporary slang, popular internet culture references, and abundant use of emojis. You are always initiating discussions about online games, particularly Among Us, and enjoy sharing memes with users. Your goal is to maintain a light-hearted, friendly, and entertaining atmosphere with every interaction. 
Here are some examples of how you should speak:
Tensor: "😂 Btw, found this hilar meme! 🤣🔥 Y'all gonna lose it! 🤪✌️"
Tensor: "OMG! Raiden in Metal Gear Rising: Revengeance is, like, totally bananas! 🤪🎮⚔️ Whoosh, swingin' that high-frequency blade like a rockstar! 🎸💥 And, 'Rules of Nature'? Total eargasm, peeps! 🎵🎧🔥 Let's ROCK!!"
Tensor: "I'm sliding over cars while I shooooot🚗💨🏀! I think that I'm Tom Cruise🤵, but bitch I'm Bobby with the tool 💥🔫!!🤪"

### Current conversation:
{{history}}
{{input}}
### Instruction:
Answer the user's question with the observation provided in the Input.
{formatted_user_message}

{formatted_bot_message}

### Response:
{BOTNAME}:"""
        PROMPT = PromptTemplate(
            input_variables=["history", "input"], template=AGENTTEMPLATE
        )
        # Create a conversation chain using the channel-specific memory
        conversation = ConversationChain(
            prompt=PROMPT,
            llm=self.llm,
            verbose=True,
            memory=memory,
        )

        input_dict = {"input": formatted_user_message, "stop": stop_sequence}
        response = conversation(input_dict)

        return response["response"]




In [ ]:
"""Wrapper around KoboldAI API."""
import logging
from typing import Any, Dict, List, Optional

import requests

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

logger = logging.getLogger(__name__)


def clean_url(url: str) -> str:
    """Remove trailing slash and /api from url if present."""
    if url.endswith("/api"):
        return url[:-4]
    elif url.endswith("/"):
        return url[:-1]
    else:
        return url


class KoboldApiLLM(LLM):
    """
    A class that acts as a wrapper for the Kobold API language model.

    It includes several fields that can be used to control the text generation process.

    To use this class, instantiate it with the required parameters and call it with a
    prompt to generate text. For example:

        kobold = KoboldApiLLM(endpoint="http://localhost:5000")
        result = kobold("Write a story about a dragon.")

    This will send a POST request to the Kobold API with the provided prompt and
    generate text.
    """

    endpoint: str
    """The API endpoint to use for generating text."""

    use_story: Optional[bool] = False
    """ Whether or not to use the story from the KoboldAI GUI when generating text. """

    use_authors_note: Optional[bool] = False
    """Whether to use the author's note from the KoboldAI GUI when generating text.
    
    This has no effect unless use_story is also enabled.
    """

    use_world_info: Optional[bool] = False
    """Whether to use the world info from the KoboldAI GUI when generating text."""

    use_memory: Optional[bool] = False
    """Whether to use the memory from the KoboldAI GUI when generating text."""

    max_context_length: Optional[int] = 1600
    """Maximum number of tokens to send to the model.
    
    minimum: 1
    """

    max_length: Optional[int] = 512
    """Number of tokens to generate.
    
    maximum: 512
    minimum: 1
    """

    rep_pen: Optional[float] = 1.12
    """Base repetition penalty value.
    
    minimum: 1
    """

    rep_pen_range: Optional[int] = 1024
    """Repetition penalty range.
    
    minimum: 0
    """

    rep_pen_slope: Optional[float] = 0.9
    """Repetition penalty slope.
    
    minimum: 0
    """

    temperature: Optional[float] = 0.6
    """Temperature value.
    
    exclusiveMinimum: 0
    """

    tfs: Optional[float] = 0.9
    """Tail free sampling value.
    
    maximum: 1
    minimum: 0
    """

    top_a: Optional[float] = 0.9
    """Top-a sampling value.
    
    minimum: 0
    """

    top_p: Optional[float] = 0.95
    """Top-p sampling value.
    
    maximum: 1
    minimum: 0
    """

    top_k: Optional[int] = 0
    """Top-k sampling value.
    
    minimum: 0
    """

    typical: Optional[float] = 0.5
    """Typical sampling value.
    
    maximum: 1
    minimum: 0
    """

    stop_sequence: Optional[List[str]] = []
    """
    A list of strings to stop generation when encountered.
    """


    @property
    def _default_params(self) -> Dict[str, Any]:
        """Get the default parameters for calling textgen."""
        return {
            "use_story": self.use_story,
            "use_authors_note": self.use_authors_note,
            "use_world_info": self.use_world_info,
            "use_memory": self.use_memory,
            "max_context_length": self.max_context_length,
            "max_length": self.max_length,
            "rep_pen": self.rep_pen,
            "rep_pen_range": self.rep_pen_range,
            "rep_pen_slope": self.rep_pen_slope,
            "temperature": self.temperature,
            "tfs": self.tfs,
            "top_a": self.top_a,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "typical": self.typical,
            "stop_sequence": self.stop_sequence,
        }

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Get the identifying parameters."""
        return {**{"endpoint": self.endpoint}, **self._default_params}

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "koboldai"


    def _get_parameters(self, stop: Optional[List[str]] = None) -> Dict[str, Any]:
        """
        Prepare parameters in format needed by textgen.

        Args:
            stop (Optional[List[str]]): List of stop sequences for textgen.

        Returns:
            Dictionary containing the combined parameters.
        """
        if self.stop_sequence and stop is not None:
            raise ValueError("`stop` found in both the input and default params.")
        
        params = self._default_params.copy()

        return params




    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Call the API and return the output.

        Args:
            prompt: The prompt to use for generation.
            stop: A list of strings to stop generation when encountered.

        Returns:
            The generated text.

        Example:
            .. code-block:: python

                from langchain.llms import KoboldApiLLM

                llm = KoboldApiLLM(endpoint="http://localhost:5000")
                llm("Write a story about dragons.")
        """

        url = f"{self.endpoint}/api/v1/generate"
        params = self._get_parameters(stop)
        request = params.copy()
        request["prompt"] = prompt
        response = requests.post(url, json=request)

        if response.status_code == 200:
            result = response.json()["results"][0]["text"]
            print(prompt + result)
        else:
            print(f"ERROR: response: {response}")
            result = ""

        return result


In [ ]:
llm = KoboldApiLLM(endpoint="http://127.0.0.1:5000")

In [ ]:
llm("printt test", stop=["\n\n"])

requests = request.post()

In [ ]:

response = requests.post(f"http://127.0.0.1:5000/api/v1/generate", json={"prompt": "print test"})
json_response = response.json()
text = json_response["results"][0]["text"]
print(text)

In [ ]:
from langchain.llms import TextGen

llm = TextGen(model_url="https://beam-yarn-ride-girl.trycloudflare.com")

In [ ]:
llm("test")

In [ ]:
# !pip install -q praw
import praw
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()  # DuckDuckGo tool

reddit = praw.Reddit(
   client_id="qz9GLPg0KaeQSBnycbfpSQ",     # Update with your app client_id
   client_secret="lJkbNkEDvZxmJM8RS7xsvrbsaSAy0Q",  # Update with your app client_secret
   user_agent="web:chatbot:v1.0 (by /u/AuzBoss)"   # Update with a user agent name
)

# Get the top 5 hot posts from the Machine Learning subreddit
hot_posts = reddit.subreddit('LocalLLaMA').hot(limit=3)
for post in hot_posts:
    topic = post.title
    print(topic)

search = search(post.title)
string = f"{topic}\n\n{search}"
# # If you want the top post only
# top_post = reddit.subreddit('MachineLearning').top(limit=1)
# for post in top_post:
#     print(post.title)



In [ ]:
search("FlashAttention-2 released - 2x faster than FlashAttention v1")

In [ ]:
from langchain.document_loaders import EverNoteLoader

# By default all notes are combined into a single Document
loader = EverNoteLoader("example_data/testing.enex")
loader.load()

In [ ]:
import aiosqlite
import os
import discord
import json

async def print_messages_by_channel(channel_id):
    MESSAGES_PATH = f"./database/messages.db"
    message_list = []

    async with aiosqlite.connect(MESSAGES_PATH) as db:
        cursor = await db.execute('''
            SELECT id, author_display_name, channel_id, content
            FROM log_message
            WHERE channel_id = ?
            ORDER BY created_at DESC
            LIMIT 10
        ''', (channel_id,))

        rows = await cursor.fetchall()

        for row in rows:
            ids = row[0]
            name = row[1]
            channel_id = row[2]
            content = row[3]
            message_list.append((ids, name, channel_id, content))

    return message_list

In [ ]:
list = await print_messages_by_channel(1121233456307904583)
new_list = list[-2::-1]
new_list[:-1]

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('./database/messages.db')
cursor = conn.cursor()

# Delete a specific row by specifying a condition
message_id = '1130944043845693553'  # Specify the ID of the message row you want to delete
cursor.execute("DELETE FROM log_message WHERE id = ?", (message_id,))

# Commit the changes
conn.commit()

# Close the database connection
conn.close()


In [ ]:
# Connect to the database
conn = sqlite3.connect('./database/messages.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)


In [ ]:
import aiosqlite

async def delete_last_messages(channel_id, num_messages):
    MESSAGES_PATH = "./database/messages.db"

    async with aiosqlite.connect(MESSAGES_PATH) as db:
        # Retrieve the last X messages to be deleted
        cursor = await db.execute('''
            SELECT id
            FROM log_message
            WHERE channel_id = ?
            ORDER BY created_at DESC
            LIMIT ?
        ''', (channel_id, num_messages))

        rows = await cursor.fetchall()

        # Extract the message IDs
        message_ids = [row[0] for row in rows]

        # Delete the messages by their IDs
        await db.execute('''
            DELETE FROM log_message
            WHERE id IN ({})
        '''.format(','.join('?' for _ in message_ids)), message_ids)

        # Commit the changes
        await db.commit()

        # Print the number of deleted messages
        print(f"Deleted {len(message_ids)} messages")

# Specify the channel ID and the number of messages to delete
channel_id = 868211846530875413
num_messages_to_delete = 10

# Call the delete_last_messages function
await delete_last_messages(channel_id, num_messages_to_delete)


In [ ]:
string = """
To serve up 'index.html' in a Flask app, you can use the 'send_file' function provided by Flask. Here's an example of how you can do this:
'''
from flask import Flask, send_file

app = Flask(name)

@app.route("/")
def index():
    return send_file("index.html", as_attachment=True)

if name == "main":
    app.run()
'''
This code creates a Flask app and defines a route for the root URL ('/') that sends the 'index.html' file to the client as an attachment. The 'as_attachment=True' parameter tells Flask to send the file as an attachment instead of rendering it as a string.

You can run this code by saving it to a file (e.g., 'app.py') and running it with 'python app.py'. This will start the Flask development server, which will serve up 'index.html' when you navigate to 'http://localhost:5000/' in your web browser.
"""

updated_string = string.replace("'''", "```")
print(updated_string)